In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from Indexing_function.Indexing_data import create_indexingData
from Indexing_function.faiss_index import create_FaissIndexing
from Indexing_function.bm25_index import create_BM25Indexing

In [3]:
# กำหนด root directory หลักของโปรเจค
ROOT_DIRECTORY = "D:\Data_sci_internship\Exploring Generative AI for Predictive Maintenance Applications"

# กำหนดชื่อโฟลเดอร์ย่อยต่างๆ
PROJECT_DIRECTORY = "predictive-maintenance-chatbot"    # โฟลเดอร์โปรเจค
DATA_ROOT_DIRECTORY = "data"                            # โฟลเดอร์หลักสำหรับเก็บข้อมูล
PREPARED_DATA_DIRECTORY = "prepared_data"               # โฟลเดอร์สำหรับข้อมูลที่ประมวลผลแล้ว

BUILDING_ROOT_DIRECTORY = "building-knowledge-base"
BUILD_NAME_DIRECTOR = "build_1"

# สร้าง path เต็มสำหรับโฟลเดอร์ข้อมูลที่ประมวลผลแล้ว
prepared_data_dir = os.path.join(
    ROOT_DIRECTORY, PROJECT_DIRECTORY, DATA_ROOT_DIRECTORY, PREPARED_DATA_DIRECTORY
)

buildingBase_data_dir = os.path.join(
    ROOT_DIRECTORY, PROJECT_DIRECTORY, DATA_ROOT_DIRECTORY, BUILDING_ROOT_DIRECTORY, BUILD_NAME_DIRECTOR
)

In [4]:
# เลือกไฟส์ข้อมูลเพื่อนำไปสร้าง Indexing ที่ต้องการใน prepared_data 
input_files = [
    # "PLANT_01_COMP_SG01_chunks_contextualized.json",
    # "PLANT_02_GEN_DF_01_chunks_contextualized.json",
    # "PLANT_02_GEN_DF_02_chunks_contextualized.json",
    # "PLANT_02_PMP_WI_01_chunks_contextualized.json",
    # "PLANT_02_PMP_WI_02_chunks_contextualized.json",
    "PLANT_02_PMP_WI_03_chunks_contextualized.json",
    "PLANT_02_SYS_WP_01_chunks_contextualized.json",
    ]

# Call the pipeline_numeric2text function
index_data = create_indexingData(prepared_data_dir, input_files, buildingBase_data_dir)

Save the Merged data JSON Complete ✔️ - Path: D:\Data_sci_internship\Exploring Generative AI for Predictive Maintenance Applications\predictive-maintenance-chatbot\data\building-knowledge-base\build_1\merged.json
Save the Index data JSON Complete ✔️ - Path: D:\Data_sci_internship\Exploring Generative AI for Predictive Maintenance Applications\predictive-maintenance-chatbot\data\building-knowledge-base\build_1\index_data.json


In [5]:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
model_name = "sentence-transformers/all-mpnet-base-v2"
create_FaissIndexing(index_data, model_name, buildingBase_data_dir)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Saved vector store Complete ✔️ - To: D:\Data_sci_internship\Exploring Generative AI for Predictive Maintenance Applications\predictive-maintenance-chatbot\data\building-knowledge-base\build_1\faiss_index\contextualized_index
Saved vector store Complete ✔️ - To: D:\Data_sci_internship\Exploring Generative AI for Predictive Maintenance Applications\predictive-maintenance-chatbot\data\building-knowledge-base\build_1\faiss_index\originalcontext_index


In [6]:
create_BM25Indexing(index_data, buildingBase_data_dir)

Saved BM25 store Complete ✔️ - To: D:\Data_sci_internship\Exploring Generative AI for Predictive Maintenance Applications\predictive-maintenance-chatbot\data\building-knowledge-base\build_1\bm25_index\contextualized_index
Saved BM25 store Complete ✔️ - To: D:\Data_sci_internship\Exploring Generative AI for Predictive Maintenance Applications\predictive-maintenance-chatbot\data\building-knowledge-base\build_1\bm25_index\originalcontext_index


---


In [ ]:
original_bm25 = create_BM25Indexing(index_data, buildingBase_data_dir)

In [11]:
import pickle


bm25_path = os.path.join(buildingBase_data_dir, "bm25_index", "contextualized_index", "BM25Model.pkl")

with open(bm25_path, "rb") as f:
    loaded_bm25 = pickle.load(f)
print("Model loaded successfully!")

Model loaded successfully!


In [12]:
import re


def tokenization(text):
    """Tokenizes text by lowering case, removing unwanted characters, and splitting by spaces."""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove unwanted characters
    return text.split()
 
query = "The MOTOR component of the Produce Water Injection Pump C (Tag: P-3410C) is monitored by the NDE_VIB (Motor vibration NDE) condition model, which indicates a consistent 100.0% health score with minimal 0.15% daily, weekly, and monthly changes. This suggests the motor's non-drive end vibration levels are stable and within acceptable limits, contributing to the overall 100.0% performance and condition health of the pump system.\n\nThe MOTOR component of Produce Water Injection Pump C (Tag: P-3410C) in the Everflow Utility Plant is monitored by the model NDE_VIB(Motor vibration NDE), which is a CONDITION model of INDIVIDUAL class. This model has a health score of 100.0%, with health changes recorded at 0.15% daily, 0.15% weekly, and 0.15% monthly."

tokenized_query = tokenization(query)
doc_scores =loaded_bm25.get_top_n(tokenized_query, corpus, n=1)
doc_top = loaded_bm25.get_scores(tokenized_query)
print(doc_scores)
print(doc_top)